# sim plan

* for every round
  * procedure?
        * if yes: 
            * offer to random resident (and choose intern who would have gotten it either way for log)
                * if accepts: increment and log to both
                * else: log miss only
        * else
            * log missed procedure0
    * 

# first attempt with no probability:

In [42]:
import numpy as np
import pandas as pd
from icecream import ic

In [76]:
residents = pd.read_excel("residents.xlsx")
residents

,name,training_level,paracentesis_completed
0,R2 1,senior,0
1,R2 2,senior,0
2,R2 3,senior,0
3,R2 4,senior,0
4,R2 5,senior,0
5,R2 6,senior,0
6,R2 7,senior,0
7,R2 8,senior,0
8,R2 9,senior,0
9,R2 10,senior,0


## note no need to record "signed off" state:

In [77]:
# no need to record "signed off"; can be generated at will
(residents.paracentesis_completed >= 5).rename("signed off")

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10     True
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
Name: signed off, dtype: bool

## anyway:

In [78]:
seniors = residents.training_level == "senior"
residents[seniors]

,name,training_level,paracentesis_completed
0,R2 1,senior,0
1,R2 2,senior,0
2,R2 3,senior,0
3,R2 4,senior,0
4,R2 5,senior,0
5,R2 6,senior,0
6,R2 7,senior,0
7,R2 8,senior,0
8,R2 9,senior,0
9,R2 10,senior,0


In [79]:
interns = residents.training_level == "intern"
residents[interns]

,name,training_level,paracentesis_completed
20,R1 1,intern,0
21,R1 2,intern,1
22,R1 3,intern,0
23,R1 4,intern,0
24,R1 5,intern,0
25,R1 6,intern,0
26,R1 7,intern,0
27,R1 8,intern,0
28,R1 9,intern,0
29,R1 10,intern,0


In [80]:
d = residents[seniors].to_dict("index")

In [81]:
np.random.choice(d)

ValueError: a must be 1-dimensional or an integer

In [106]:
residents[seniors].sample().name.values[0]

'R2 2'

0      True
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
Name: name, dtype: bool

In [105]:
residents.sample().name.values[0]

'R1 4'

In [93]:
residents.loc[:, "name"]

0      R2 1
1      R2 2
2      R2 3
3      R2 4
4      R2 5
5      R2 6
6      R2 7
7      R2 8
8      R2 9
9     R2 10
10     R3 1
11     R3 2
12     R3 3
13     R3 4
14     R3 5
15     R3 6
16     R3 7
17     R3 8
18     R3 9
19    R3 10
20     R1 1
21     R1 2
22     R1 3
23     R1 4
24     R1 5
25     R1 6
26     R1 7
27     R1 8
28     R1 9
29    R1 10
Name: name, dtype: object

In [ ]:
residents[seniors].sample().name.values[0]

In [117]:
residents[residents.name == "R3 1"].paracentesis_completed.values[0] >= 1

np.True_

In [196]:
days = 1
signoff_threshold = 5
signed_off_avidity = 0.8
not_signed_off_avidity = 0.1

for day in range(days):
    procedure_opportunity_today = True
    selected_senior = ic(residents[seniors].sample().name.values[0])
    ic(residents[residents.name == selected_senior].paracentesis_completed.values[0])
    selected_intern = ic(residents[interns].sample().name.values[0])
    ic(residents[residents.name == selected_intern].paracentesis_completed.values[0])

    if ic((residents[residents.name == selected_senior].paracentesis_completed.values[0] >= 1)):
        ic("signed off:")
        ic(residents[residents.name == selected_senior])
        ic(residents[residents.name == selected_intern])
        if ic(np.random.rand() <= signed_off_avidity):
            ic("procedure done")
            residents.loc[residents.name == selected_senior,"paracentesis_completed"] += 1
            residents.loc[residents.name == selected_intern,"paracentesis_completed"] += 1
        else:
            ic("no procedure")
        ic(residents[residents.name == selected_senior])
        ic(residents[residents.name == selected_intern])
    else: # not signed off
        ic("not signed off:")
        ic(residents[residents.name == selected_senior])
        ic(residents[residents.name == selected_intern])
        if ic(np.random.rand() <= not_signed_off_avidity):
            ic("procedure done")
            residents.loc[residents.name == selected_senior,"paracentesis_completed"] += 1
            residents.loc[residents.name == selected_intern,"paracentesis_completed"] += 1
        else:
            ic("no procedure")
        ic(residents[residents.name == selected_senior])
        ic(residents[residents.name == selected_intern])

ic| residents[seniors].sample().name.values[0]: 'R3 4'
ic| residents[residents.name == selected_senior].paracentesis_completed.values[0]: np.int64(1)
ic| residents[interns].sample().name.values[0]: 'R1 10'
ic| residents[residents.name == selected_intern].paracentesis_completed.values[0]: np.int64(0)
ic| residents[residents.name == selected_senior].paracentesis_completed.values[0] >= 1: np.True_
ic| 'signed off:'
ic| residents[residents.name == selected_senior]:     name training_level  paracentesis_completed
                                                  13  R3 4         senior                       1
ic| residents[residents.name == selected_intern]:      name training_level  paracentesis_completed
                                                  29  R1 10         intern                       0
ic| np.random.rand() <= signed_off_avidity: True
ic| 'procedure done'
ic| residents[residents.name == selected_senior]:     name training_level  paracentesis_completed
                      